# Pix2Pixで宅地利用調査から任意の衛星画像を生成するアルゴリズムを作る


今回は宙畑でも何回か取り上げられている pix2pix という画像生成手法を利用していきます。
画像は国土地理院の宅地利用動向調査成果のデータを利用します。これは、都市圏の土地利用データで、1974年から1997年の間のデータは地理院タイルとして提供されています。今回は近畿圏の1991年のものを利用しました。

同じく地理院タイルでは、年代別の写真として、1987年～1990年の上空の写真が提供されているので、同じタイル座標の宅地利用画像を入力画像として、出力を上空の写真にします。

コードを実行するにあたり、教師データが必要となります。
教師データは以下のリンクからダウンロードしてください。

In [ ]:
# pix2pix を実行できるようにするためにコードを持ってきます
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2447, done.
remote: Total 2447 (delta 0), reused 0 (delta 0), pack-reused 2447
Receiving objects: 100% (2447/2447), 8.18 MiB | 26.42 MiB/s, done.
Resolving deltas: 100% (1535/1535), done.


In [ ]:
# ダウンロードしたディレクトリを利用するようにします
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
# pix2pixで必要になるライブラリをインストールします
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 676 kB 8.0 MB/s 
     |████████████████████████████████| 1.8 MB 52.5 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 145 kB 68.0 MB/s 
     |████████████████████████████████| 181 kB 68.7 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=679bca7c953c354606442e4d5f199e98bf25f783525f74d1a6901872abf978a7
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c2370536d8bece781dddb4754366695716e358a460c27ff397ffa8b6a4aa3ac6
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
  Created wheel for torchfile: filen

In [ ]:
# 学習用データを生成します
!python datasets/make_dataset_aligned.py --dataset-path /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat

/content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat


In [ ]:
# 学習を行います
# 学習する回数が多いほど、結果を得るために時間を要します
# 計算時間が長いために処理が途中で止まってしまう場合には、--n_epochs 50 --n_epochs_decay 10の数字を小さくするようにしてください
!python train.py --dataroot /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat --name map2sat --model pix2pix --batch_size 64 --gpu_ids 0 --checkpoints_dir /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat_checkpoints --direction BtoA --n_epochs 50 --n_epochs_decay 10

----------------- Options ---------------
               batch_size: 64                            	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat_checkpoints	[default: ./checkpoints]
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat	[default: None]
             dataset_mode: aligned                       
                direction: BtoA                          	[default: AtoB]
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
  

In [ ]:
# 学習時には使われていなかったテスト画像でデータを生成します
!python test.py --dataroot /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat --name map2sat --model pix2pix --gpu_ids 0 --checkpoints_dir /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat_checkpoints --direction BtoA --results_dir /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat_results

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat_checkpoints	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat	[default: None]
             dataset_mode: aligned                       
                direction: BtoA                          	[default: AtoB]
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
               

In [ ]:
# オリジナルで生成した地図を元にした画像を生成します
!python test.py --dataroot /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat_original/ --name map2sat_original --model test --checkpoints_dir /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat_checkpoints --direction BtoA --results_dir /content/drive/MyDrive/sorabatake/12_contents/Pix2Pix/map2sat_results --dataset_mode single --netG unet_256 --norm batch